# Selección del mejor modelo de regresión
En este notebook se presentan los pasos para seleccionar el mejor modelo de regresión para el conjunto de datos de credenciales. Los modelos que se evaluarán son:
- [Regresión lineal](https://scikit-learn.org/stable/modules/linear_model.html#ordinary-least-squares)
- [K-vecinos más cercanos](https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-regression)
- [Árboles de decisión](https://scikit-learn.org/stable/modules/tree.html#regression)
- [Random Forest](https://scikit-learn.org/stable/modules/ensemble.html#random-forests-and-other-randomized-tree-ensembles)
- [Cresta Bayesiana](https://scikit-learn.org/stable/modules/linear_model.html#bayesian-ridge-regression)
- [Máquina de vector soporte (SVR)](https://scikit-learn.org/stable/modules/svm.html#regression)
- [Descenso de Gradiente Estocástico](https://scikit-learn.org/stable/modules/sgd.html#regression)
- [Red elástica](https://scikit-learn.org/stable/modules/linear_model.html#elastic-net)
- [Regresión de cresta de Kernel](https://scikit-learn.org/stable/modules/kernel_ridge.html)


In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import (
    LinearRegression,
    BayesianRidge,
    SGDRegressor,
    ElasticNet
)
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)
from math import sqrt
import pickle

## Lectura del dataset

In [2]:
df = pd.read_json('data/train.json', orient='records')
df

,Muestra_ID,USERNAME,VIP_credentials,VIP_group,User_status,PASSWORD,MD5,SHA256,SHA512,SHA1,...,Engagement_rate,Mentions,Posts_day,Reposts,Channel_country,FILE_NAME,Results_file,Country_file_name,TOTAL,RISK
0,1.1,pablopiedrabuena@telebot.com,Vip,Ciber,Active,-P#%1NZoH7NZ,b5262d1151b996f12737e49fa2167fbc,a4799b6642c076748357843a07c0d1bcd3e5eb75ccc0d3...,8d10e6af68a519bf194f880940bb803862320564612323...,84fe8d4c552ef52149c7fb0d80060e51275f1847,...,17.58,21,2,10,Other,Ch.txt,1,Other,73.223273,High
1,2.1,davidgarcia@telebot.com,Vip,Ciber,Active,uucp,45b9cfabd8945aa221eb15c629bbf760,730b5353b8edd9b56300b355db37cc39d9da119a68cb1e...,df0f9be2269968c7759fe4e56092f9231f457e5a96a968...,a2dd138f615b6351bd5307c991d6790b5e8c5d9b,...,15.52,25,2,11,Other,France.txt,1,Other,73.827687,High
2,50.1,alvaroafonso@telebot.com,Vip,Ciber,Active,netamin,f350f792fe650c0f858e0cae9cc5a6c7,f860097832f116d4a69b629e712afec9bcba8d3984fec7...,162e3ffef02c08a0b8005b991b2b31db4da76d579f12d4...,0870c592f92eb3d6ba136df5ee4086dd2d0ae21a,...,19.61,21,3,7,Other,Europe.txt,1,Europe,72.340851,High
3,3.1,miguelchacon@telebot.com,Vip,Ciber,Active,pepe,926e27eecdbc7a18858b3798ba99bddd,7c9e7c1494b2684ab7c19d6aff737e460fa9e98d5a234d...,974f3036f39834082e23f4d70f1feba9d4805b3ee2cedb...,265392dc2782778664cc9d56c8e3cd9956661bb0,...,1.00,1,1,1,Other,900K.txt,1,Other,71.606993,High
4,4.1,eduardobejarano@telebot.com,Vip,Ciber,Active,WrMD4fw7,33763d048d98dc38fe9c100067fae7ca,13da9707d7e30adcfa062bf32d8a51219408d86caf5665...,464304b29b44eca6c123853a57b4759268aa2383bb08af...,99cf2501581ed28d097847282e50fab8b5df53b3,...,18.89,68,13,14,Other,44K.txt,1,Other,75.450268,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,52.3,rafaelortega@telebot.com,Vip,Ciber,Active,d9nW6oGu,be3f6126764ad36f785f329fea894628,b9896ebab1da8541dbf5cf7c344289b35166fdc0b7a10c...,4f51cea0cb40010c48b2a36a78d56a26eb1639c23c55a3...,70dc35eacc4150e8f3944d6a8aef0983c4ebe055,...,17.12,161,4,4,Other,UPL.txt,8,Other,77.449543,High
193,52.4,pablopiedrabuena@telebot.com,Vip,Ciber,Active,Telebot2023+,4e1afb717d0492f6058984ac34023c16,d6affb3ed382f48e33f496003197dd349d074a11256376...,c657a107790bda8e49f1dd730c8aad929c1a1a6986b4fa...,76852cb8cd03c0df456c821eeb031e0ce01d0576,...,32.57,3,2,3,Other,Portugal.txt,8,Other,74.334728,High
194,52.5,albertoestirado@telebot.com,Vip,Ciber,Active,bee,9dfd70fdf15a3cb1ea00d7799ac6651b,62cb81b5904a262ffaeed02abef36bfc540b09f964b8b0...,0159d8667999bcc9e752655a43359c1eaf8958c519faa5...,917ee46db0cda4c4739bb176b3cf68880f7dba95,...,16.30,74,12,14,Other,900K.txt,8,Other,79.371522,High
195,52.6,mariadiaz@telebot.com,Vip,Ciber,Active,12345,e2f91f170f15390b5084510e546c09f8,bc47ae33866b036dd2d3c9fa53ce34786087d047c830c8...,be4c43b9eb51e52a4c54fb153f9f13f2f07969adf56ce1...,1e628815f4ecb1c23e66bdfb8252db752107989e,...,23.13,260,9,3,Other,UPL.txt,8,Other,78.940622,High


## Preprocesamiento de datos

In [3]:
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
df.columns = df.columns.str.lower()
df['channel'] = df['channel_name']
df['file'] = df['file_name']
#keep
columns = [
    "username",
    "password",
    "channel",
    "file",
    "vip_credentials",
    "vip_group",
    "user_status",
    "md5",
    "sha256",
    "sha512",
    "sha1",
    "password_update",
    "password_type",
    "leaked_password",
    "password_strength",
    "guesses_discover",
    "cracking_time",
    "password_entropy",
    "chat_type",
    "channel_privacity",
    "subscribers",
    "engagement_rate",
    "mentions",
    "posts_day",
    "reposts",
    "channel_country",
    "country_file_name",
    'total'
]
df['vip_credentials'] = df['vip_credentials'].str.lower()
df['vip_group'] = df['vip_group'].str.lower()
df['user_status'] = df['user_status'].str.lower()
df['password_type'] = df['password_type'].str.lower()
df['password_update'] = df['password_update'].str.lower()
df = df[columns]
df

,username,password,channel,file,vip_credentials,vip_group,user_status,md5,sha256,sha512,...,chat_type,channel_privacity,subscribers,engagement_rate,mentions,posts_day,reposts,channel_country,country_file_name,total
0,pablopiedrabuena@telebot.com,-P#%1NZoH7NZ,Cloudata FREE Databases ComboLists,Ch.txt,vip,ciber,active,b5262d1151b996f12737e49fa2167fbc,a4799b6642c076748357843a07c0d1bcd3e5eb75ccc0d3...,8d10e6af68a519bf194f880940bb803862320564612323...,...,Channel,Public,8845,17.58,21,2,10,Other,Other,73.223273
1,davidgarcia@telebot.com,uucp,GODELESS CLOUD,France.txt,vip,ciber,active,45b9cfabd8945aa221eb15c629bbf760,730b5353b8edd9b56300b355db37cc39d9da119a68cb1e...,df0f9be2269968c7759fe4e56092f9231f457e5a96a968...,...,Channel,Public,9393,15.52,25,2,11,Other,Other,73.827687
2,alvaroafonso@telebot.com,netamin,Group,Europe.txt,vip,ciber,active,f350f792fe650c0f858e0cae9cc5a6c7,f860097832f116d4a69b629e712afec9bcba8d3984fec7...,162e3ffef02c08a0b8005b991b2b31db4da76d579f12d4...,...,Channel,Public,2340,19.61,21,3,7,Other,Europe,72.340851
3,miguelchacon@telebot.com,pepe,NoName_Cloud,900K.txt,vip,ciber,active,926e27eecdbc7a18858b3798ba99bddd,7c9e7c1494b2684ab7c19d6aff737e460fa9e98d5a234d...,974f3036f39834082e23f4d70f1feba9d4805b3ee2cedb...,...,Channel,Private,508,1.00,1,1,1,Other,Other,71.606993
4,eduardobejarano@telebot.com,WrMD4fw7,[ REBREND ],44K.txt,vip,ciber,active,33763d048d98dc38fe9c100067fae7ca,13da9707d7e30adcfa062bf32d8a51219408d86caf5665...,464304b29b44eca6c123853a57b4759268aa2383bb08af...,...,Channel,Public,7852,18.89,68,13,14,Other,Other,75.450268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,rafaelortega@telebot.com,d9nW6oGu,DB-MARKEET,UPL.txt,vip,ciber,active,be3f6126764ad36f785f329fea894628,b9896ebab1da8541dbf5cf7c344289b35166fdc0b7a10c...,4f51cea0cb40010c48b2a36a78d56a26eb1639c23c55a3...,...,Channel,Public,5096,17.12,161,4,4,Other,Other,77.449543
193,pablopiedrabuena@telebot.com,Telebot2023+,Databases World🔰,Portugal.txt,vip,ciber,active,4e1afb717d0492f6058984ac34023c16,d6affb3ed382f48e33f496003197dd349d074a11256376...,c657a107790bda8e49f1dd730c8aad929c1a1a6986b4fa...,...,Channel,Public,2244,32.57,3,2,3,Other,Other,74.334728
194,albertoestirado@telebot.com,bee,[ REBREND ],900K.txt,vip,ciber,active,9dfd70fdf15a3cb1ea00d7799ac6651b,62cb81b5904a262ffaeed02abef36bfc540b09f964b8b0...,0159d8667999bcc9e752655a43359c1eaf8958c519faa5...,...,Channel,Public,8129,16.30,74,12,14,Other,Other,79.371522
195,mariadiaz@telebot.com,12345,OPENBULLET,UPL.txt,vip,ciber,active,e2f91f170f15390b5084510e546c09f8,bc47ae33866b036dd2d3c9fa53ce34786087d047c830c8...,be4c43b9eb51e52a4c54fb153f9f13f2f07969adf56ce1...,...,Channel,Public,4974,23.13,260,9,3,Other,Other,78.940622


In [4]:
X = df.drop(columns=['total'])
X

,username,password,channel,file,vip_credentials,vip_group,user_status,md5,sha256,sha512,...,password_entropy,chat_type,channel_privacity,subscribers,engagement_rate,mentions,posts_day,reposts,channel_country,country_file_name
0,pablopiedrabuena@telebot.com,-P#%1NZoH7NZ,Cloudata FREE Databases ComboLists,Ch.txt,vip,ciber,active,b5262d1151b996f12737e49fa2167fbc,a4799b6642c076748357843a07c0d1bcd3e5eb75ccc0d3...,8d10e6af68a519bf194f880940bb803862320564612323...,...,78.66,Channel,Public,8845,17.58,21,2,10,Other,Other
1,davidgarcia@telebot.com,uucp,GODELESS CLOUD,France.txt,vip,ciber,active,45b9cfabd8945aa221eb15c629bbf760,730b5353b8edd9b56300b355db37cc39d9da119a68cb1e...,df0f9be2269968c7759fe4e56092f9231f457e5a96a968...,...,18.80,Channel,Public,9393,15.52,25,2,11,Other,Other
2,alvaroafonso@telebot.com,netamin,Group,Europe.txt,vip,ciber,active,f350f792fe650c0f858e0cae9cc5a6c7,f860097832f116d4a69b629e712afec9bcba8d3984fec7...,162e3ffef02c08a0b8005b991b2b31db4da76d579f12d4...,...,32.90,Channel,Public,2340,19.61,21,3,7,Other,Europe
3,miguelchacon@telebot.com,pepe,NoName_Cloud,900K.txt,vip,ciber,active,926e27eecdbc7a18858b3798ba99bddd,7c9e7c1494b2684ab7c19d6aff737e460fa9e98d5a234d...,974f3036f39834082e23f4d70f1feba9d4805b3ee2cedb...,...,18.80,Channel,Private,508,1.00,1,1,1,Other,Other
4,eduardobejarano@telebot.com,WrMD4fw7,[ REBREND ],44K.txt,vip,ciber,active,33763d048d98dc38fe9c100067fae7ca,13da9707d7e30adcfa062bf32d8a51219408d86caf5665...,464304b29b44eca6c123853a57b4759268aa2383bb08af...,...,47.63,Channel,Public,7852,18.89,68,13,14,Other,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,rafaelortega@telebot.com,d9nW6oGu,DB-MARKEET,UPL.txt,vip,ciber,active,be3f6126764ad36f785f329fea894628,b9896ebab1da8541dbf5cf7c344289b35166fdc0b7a10c...,4f51cea0cb40010c48b2a36a78d56a26eb1639c23c55a3...,...,47.63,Channel,Public,5096,17.12,161,4,4,Other,Other
193,pablopiedrabuena@telebot.com,Telebot2023+,Databases World🔰,Portugal.txt,vip,ciber,active,4e1afb717d0492f6058984ac34023c16,d6affb3ed382f48e33f496003197dd349d074a11256376...,c657a107790bda8e49f1dd730c8aad929c1a1a6986b4fa...,...,78.66,Channel,Public,2244,32.57,3,2,3,Other,Other
194,albertoestirado@telebot.com,bee,[ REBREND ],900K.txt,vip,ciber,active,9dfd70fdf15a3cb1ea00d7799ac6651b,62cb81b5904a262ffaeed02abef36bfc540b09f964b8b0...,0159d8667999bcc9e752655a43359c1eaf8958c519faa5...,...,14.10,Channel,Public,8129,16.30,74,12,14,Other,Other
195,mariadiaz@telebot.com,12345,OPENBULLET,UPL.txt,vip,ciber,active,e2f91f170f15390b5084510e546c09f8,bc47ae33866b036dd2d3c9fa53ce34786087d047c830c8...,be4c43b9eb51e52a4c54fb153f9f13f2f07969adf56ce1...,...,16.51,Channel,Public,4974,23.13,260,9,3,Other,Other


In [5]:
y = df['total']
y = y.astype('float64').apply(lambda x: x/100)
y

0      0.732233
1      0.738277
2      0.723409
3      0.716070
4      0.754503
         ...   
192    0.774495
193    0.743347
194    0.793715
195    0.789406
196    0.572057
Name: total, Length: 197, dtype: float64

Eliminación de características innecesarias

In [6]:
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
unnecesary_features = ['username', 'password', 'md5', 'sha256', 'sha512', 'sha1', 'channel', 'file']
categorical_features = X.select_dtypes(include=['object']).columns
# filtrar características categoricas
categorical_features = [col for col in categorical_features if col not in unnecesary_features]
# filtrar características numericas
numerical_features = [col for col in numerical_features if col not in unnecesary_features]

In [7]:
X = X.drop(columns=unnecesary_features)
X

,vip_credentials,vip_group,user_status,password_update,password_type,leaked_password,password_strength,guesses_discover,cracking_time,password_entropy,chat_type,channel_privacity,subscribers,engagement_rate,mentions,posts_day,reposts,channel_country,country_file_name
0,vip,ciber,active,not actual,personal password,0,4,12.00,1.000000e+11,78.66,Channel,Public,8845,17.58,21,2,10,Other,Other
1,vip,ciber,active,not actual,personal password,2000,1,4.00,1.000100e+03,18.80,Channel,Public,9393,15.52,25,2,11,Other,Other
2,vip,ciber,active,not actual,personal password,7,2,6.62,4.144000e+05,32.90,Channel,Public,2340,19.61,21,3,7,Other,Europe
3,vip,ciber,active,not actual,personal password,13277,0,2.31,2.030000e+01,18.80,Channel,Private,508,1.00,1,1,1,Other,Other
4,vip,ciber,active,not actual,personal password,0,2,8.00,1.000000e+07,47.63,Channel,Public,7852,18.89,68,13,14,Other,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,vip,ciber,active,not actual,personal password,0,2,8.00,1.000000e+07,47.63,Channel,Public,5096,17.12,161,4,4,Other,Other
193,vip,ciber,active,actual,default password,0,4,11.80,6.720000e+10,78.66,Channel,Public,2244,32.57,3,2,3,Other,Other
194,vip,ciber,active,not actual,personal password,4186,0,3.00,1.000000e+02,14.10,Channel,Public,8129,16.30,74,12,14,Other,Other
195,vip,ciber,active,not actual,personal password,3508324,0,0.84,7.000000e-01,16.51,Channel,Public,4974,23.13,260,9,3,Other,Other


Codificación de variables categóricas

In [8]:
ohe = OneHotEncoder(drop='if_binary', handle_unknown='ignore')
# Ajuste del codificador
ohe.fit(X[categorical_features])
# Transformación de las variables categóricas
X_ohe = ohe.transform(X[categorical_features])
# Creación de un DataFrame con las variables categóricas codificadas
X_ohe = pd.DataFrame(X_ohe.toarray(), columns=ohe.get_feature_names_out())
# Eliminación de las variables categóricas originales
X = X.drop(columns=categorical_features)
# Unión de las variables categóricas codificadas
X = pd.concat([X, X_ohe], axis=1)
X.head()

,leaked_password,password_strength,guesses_discover,cracking_time,password_entropy,subscribers,engagement_rate,mentions,posts_day,reposts,...,password_update_not actual,password_type_default password,password_type_password for change,password_type_personal password,chat_type_Group,channel_privacity_Public,channel_country_Spain,country_file_name_Europe,country_file_name_Other,country_file_name_Spain
0,0,4,12.00,1.000000e+11,78.66,8845,17.58,21,2,10,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,2000,1,4.00,1.000100e+03,18.80,9393,15.52,25,2,11,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,7,2,6.62,4.144000e+05,32.90,2340,19.61,21,3,7,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,13277,0,2.31,2.030000e+01,18.80,508,1.00,1,1,1,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,2,8.00,1.000000e+07,47.63,7852,18.89,68,13,14,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


Ajuste de características numéricas

In [9]:
scaler = MinMaxScaler()
# Ajuste del escaler
scaler.fit(X[numerical_features], y)
# Transformación de las variables numéricas
X[numerical_features] = scaler.transform(X[numerical_features])
X.head()

,leaked_password,password_strength,guesses_discover,cracking_time,password_entropy,subscribers,engagement_rate,mentions,posts_day,reposts,...,password_update_not actual,password_type_default password,password_type_password for change,password_type_personal password,chat_type_Group,channel_privacity_Public,channel_country_Spain,country_file_name_Europe,country_file_name_Other,country_file_name_Spain
0,0.000000e+00,1.00,0.795918,1.000000e-03,0.819985,0.928671,0.496050,0.056757,0.023256,0.666667,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,4.701147e-05,0.25,0.251701,9.999000e-12,0.171237,0.987539,0.437923,0.067568,0.023256,0.733333,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1.645402e-07,0.50,0.429932,4.143998e-09,0.324049,0.229885,0.553330,0.056757,0.034884,0.466667,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,3.120857e-04,0.00,0.136735,2.010000e-13,0.171237,0.033086,0.028217,0.002703,0.011628,0.066667,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.000000e+00,0.50,0.523810,1.000000e-07,0.483689,0.822000,0.533014,0.183784,0.151163,0.933333,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


## Selección de características

In [10]:
# Número de variables a seleccionar
k = 20
# Selección de variables
selector = SelectKBest(score_func=f_regression, k=k)
# Ajuste del selector
selector.fit(X, y)
# Transformación de las variables
X_selected = selector.transform(X)

In [11]:
# Obtención de los indices de las características seleccionadas
selected_feature_indices = selector.get_support(indices=True)
# Obtención de nombres de las variables de las características seleccionadas
selected_features = X.columns[selected_feature_indices]
# Subconjunto de datos con las características seleccionadas
selected_data = X[selected_features]
selected_data.head()

,password_strength,guesses_discover,cracking_time,password_entropy,subscribers,engagement_rate,mentions,posts_day,reposts,vip_credentials_vip,vip_group_ceo,vip_group_ciber,vip_group_directiva,vip_group_otros,user_status_inactive,password_type_default password,password_type_personal password,country_file_name_Europe,country_file_name_Other,country_file_name_Spain
0,1.00,0.795918,1.000000e-03,0.819985,0.928671,0.496050,0.056757,0.023256,0.666667,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.25,0.251701,9.999000e-12,0.171237,0.987539,0.437923,0.067568,0.023256,0.733333,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.50,0.429932,4.143998e-09,0.324049,0.229885,0.553330,0.056757,0.034884,0.466667,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.00,0.136735,2.010000e-13,0.171237,0.033086,0.028217,0.002703,0.011628,0.066667,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.50,0.523810,1.000000e-07,0.483689,0.822000,0.533014,0.183784,0.151163,0.933333,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [12]:
scores = selector.scores_
scores_df = pd.DataFrame({'Feature': X.columns, 'Score': scores})
# delete the label column
scores_df = scores_df.drop(scores_df[scores_df['Feature'] == 'label'].index)
scores_df = scores_df.sort_values(by='Score', ascending=False)
scores_df

,Feature,Score
15,user_status_inactive,504.351538
14,vip_group_otros,256.683894
10,vip_credentials_vip,256.683894
12,vip_group_ciber,118.144635
25,country_file_name_Spain,4.606165
1,password_strength,4.462081
4,password_entropy,4.214420
11,vip_group_ceo,3.176626
5,subscribers,2.869037
2,guesses_discover,2.821719


## División de los datos en entrenamiento y prueba

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Creación de validación cruzada
La [validación cruzada](https://scikit-learn.org/stable/modules/cross_validation.html) permite evaluar el rendimiento de los modelos de regresión en diferentes conjuntos de datos. Se utilizará la validación cruzada [KFold](https://scikit-learn.org/stable/modules/cross_validation.html#k-fold) con 5 plieges

In [14]:
# Validación cruzada
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Ajuste de hiperparámetros y selección del mejor modelo

Función para almacenar los resultados

In [15]:
model_names = []
model_scores = []
model_parameters = []

def save_results(model_name, model_score, model_best_params):
    model_names.append(model_name)
    model_scores.append(model_score)
    model_parameters.append(model_best_params)

Métricas de evaluación
- $R^2$: [Coeficiente de determinación](https://en.wikipedia.org/wiki/Coefficient_of_determination)
- [Error absoluto medio](https://en.wikipedia.org/wiki/Mean_absolute_error)
- [Error cuadrático medio](https://en.wikipedia.org/wiki/Mean_squared_error)
- Raíz del error cuadrático medio: Similar al error cuadrático medio, pero el resultado está expresado en las mismas unidades que la variable objetivo. $\sqrt{MSE}$

In [16]:
scoring = {'R2': 'r2', 'MAE': 'neg_mean_absolute_error', 'MSE': 'neg_mean_squared_error', 'RMSE': 'neg_root_mean_squared_error'}

Función para evaluar los modelos

In [17]:
def run_grid(model, params, x, y):
    grid = GridSearchCV(
        estimator=model,
        param_grid=params,
        cv=cv,
        scoring=scoring,
        n_jobs=-1,
        refit='R2',
        return_train_score=True
    )
    grid.fit(x, y)
    model = grid.get_params()['estimator']
    print(f'{"="*25} {model} {"="*25}')
    print(f'Best score: {grid.best_score_}')
    print(f'Best parameters: {grid.best_params_}')
    save_results(model, grid.best_score_, grid.best_params_)

## Regresión lineal

In [18]:
linear_regression_parameters = {}

In [19]:
run_grid(LinearRegression(), linear_regression_parameters, X_train, y_train)

========================= LinearRegression() =========================
Best score: 0.9939867829358991
Best parameters: {}


## K-vecinos más cercanos

In [20]:
knnregressor_parameters = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 20, 30, 40, 50],
}

In [21]:
run_grid(KNeighborsRegressor(), knnregressor_parameters, X_train, y_train)

========================= KNeighborsRegressor() =========================
Best score: 0.8587043270139872
Best parameters: {'algorithm': 'auto', 'leaf_size': 10, 'n_neighbors': 9, 'weights': 'distance'}


c:\Users\victorluque\Documents\Trabajo\Laboratorio\maria\.venv\lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


## Árboles de decisión

In [22]:
dtregressor_parameters = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'splitter': ['best', 'random'],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'ccp_alpha': [0.001, 0.01, 0.1, 1],
}

In [23]:
run_grid(DecisionTreeRegressor(), dtregressor_parameters, X_train, y_train)

========================= DecisionTreeRegressor() =========================
Best score: 0.9877664280129194
Best parameters: {'ccp_alpha': 0.001, 'criterion': 'absolute_error', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}


## Random Forest

In [24]:
rfregressor_parameters = {
    'n_estimators': [50, 100, 150],
    'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
    'max_depth': [10, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'ccp_alpha': [0.001, 0.01, 0.1, 1],
}
    

In [25]:
run_grid(RandomForestRegressor(), rfregressor_parameters, X_train, y_train)

========================= RandomForestRegressor() =========================
Best score: 0.9904093730794863
Best parameters: {'ccp_alpha': 0.001, 'criterion': 'absolute_error', 'max_depth': 50, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}


## Máquina de vector soporte (SVR)

In [26]:
svr_parameters = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3, 4, 5],
    'gamma': ['scale', 'auto'],
    'C': [0.01, 0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5],
}

In [27]:
run_grid(SVR(), svr_parameters, X_train, y_train)

========================= SVR() =========================
Best score: 0.9947403621745586
Best parameters: {'C': 1, 'degree': 2, 'epsilon': 0.01, 'gamma': 'scale', 'kernel': 'linear'}


## Cresta Bayesiana

In [28]:
br_parameters = {
    'tol': [0.0001, 0.001, 0.01, 0.1, 1],
    'alpha_1': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2],
    'alpha_2': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2],
    'lambda_1': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2],
    'lambda_2': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2],
}

In [29]:
run_grid(BayesianRidge(), br_parameters, X_train, y_train)

========================= BayesianRidge() =========================
Best score: 0.9952820299402412
Best parameters: {'alpha_1': 1e-06, 'alpha_2': 0.01, 'lambda_1': 0.01, 'lambda_2': 1e-06, 'tol': 0.1}


## Descenso de Gradiente Estocástico

In [30]:
sgd_parameters = {
    'loss': ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
    'l1_ratio': [0.15, 0.25, 0.5, 0.75, 0.85],
    'max_iter': [1000, 2000, 3000, 4000, 5000],
}

In [31]:
run_grid(SGDRegressor(), sgd_parameters, X_train, y_train)

========================= SGDRegressor() =========================
Best score: 0.9092060424030496
Best parameters: {'alpha': 0.0001, 'l1_ratio': 0.75, 'loss': 'epsilon_insensitive', 'max_iter': 1000, 'penalty': 'l2'}


## Red elástica

In [32]:
en_parameters = {
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
    'l1_ratio': [0.15, 0.25, 0.5, 0.75, 0.85],
    'max_iter': [1000, 2000, 3000, 4000, 5000],
}

In [33]:
run_grid(ElasticNet(), en_parameters, X_train, y_train)

========================= ElasticNet() =========================
Best score: 0.9953959646182057
Best parameters: {'alpha': 0.0001, 'l1_ratio': 0.85, 'max_iter': 1000}


## Regresión de cresta de Kernel

In [34]:
kr_parameters = {
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3, 4, 5]
}

In [35]:
run_grid(KernelRidge(), kr_parameters, X_train, y_train)

========================= KernelRidge() =========================
Best score: 0.9952058285242018
Best parameters: {'alpha': 0.01, 'degree': 2, 'kernel': 'linear'}


## Comparación de los modelos

In [36]:
# Comparación de modelos
results_df = pd.DataFrame({
    'Model': model_names,
    'Score': model_scores,
    'Parameters': model_parameters
})
results_df = results_df.sort_values(by='Score', ascending=False)
results_df = results_df.reset_index(drop=True)
results_df

,Model,Score,Parameters
0,ElasticNet(),0.995396,"{'alpha': 0.0001, 'l1_ratio': 0.85, 'max_iter'..."
1,BayesianRidge(),0.995282,"{'alpha_1': 1e-06, 'alpha_2': 0.01, 'lambda_1'..."
2,KernelRidge(),0.995206,"{'alpha': 0.01, 'degree': 2, 'kernel': 'linear'}"
3,SVR(),0.994740,"{'C': 1, 'degree': 2, 'epsilon': 0.01, 'gamma'..."
4,LinearRegression(),0.993987,{}
5,RandomForestRegressor(),0.990409,"{'ccp_alpha': 0.001, 'criterion': 'absolute_er..."
6,DecisionTreeRegressor(),0.987766,"{'ccp_alpha': 0.001, 'criterion': 'absolute_er..."
7,SGDRegressor(),0.909206,"{'alpha': 0.0001, 'l1_ratio': 0.75, 'loss': 'e..."
8,KNeighborsRegressor(),0.858704,"{'algorithm': 'auto', 'leaf_size': 10, 'n_neig..."


Hay que tener en cuenta que los resultados pueden variar en función de la semilla aleatoria utilizada para dividir los datos en entrenamiento y prueba.

## Conclusión
Los mejores modelos de regresión con un más de un 99,5% de score para el conjunto de datos de credenciales son:
- Red elástica
- Cresta Bayesiana
- Regresión de cresta de Kernel
- Máquina de vector soporte (SVR)

## Selección del mejor modelo

In [37]:
best_model_name = str(results_df['Model'][0])
print(f'Best model: {best_model_name}')
print(f'Best score: {results_df["Score"][0]}')
print(f'Best parameters: {results_df["Parameters"][0]}')

Best model: ElasticNet()
Best score: 0.9953959646182057
Best parameters: {'alpha': 0.0001, 'l1_ratio': 0.85, 'max_iter': 1000}


In [38]:
if best_model_name == 'LinearRegression()':
    best_model = LinearRegression()
elif best_model_name == 'KNeighborsRegressor()':
    best_model = KNeighborsRegressor(**results_df.iloc[0]['Parameters'])
elif best_model_name == 'DecisionTreeRegressor()':
    best_model = DecisionTreeRegressor(**results_df.iloc[0]['Parameters'])
elif best_model_name == 'RandomForestRegressor()':
    best_model = RandomForestRegressor(**results_df.iloc[0]['Parameters'])
elif best_model_name == 'SVR()':
    best_model = SVR(**results_df.iloc[0]['Parameters'])
elif best_model_name == 'BayesianRidge()':
    best_model = BayesianRidge(**results_df.iloc[0]['Parameters'])
elif best_model_name == 'SGDRegressor()':
    best_model = SGDRegressor(**results_df.iloc[0]['Parameters'])
elif best_model_name == 'ElasticNet()':
    best_model = ElasticNet(**results_df.iloc[0]['Parameters'])
elif best_model_name == 'KernelRidge()':
    best_model = KernelRidge(**results_df.iloc[0]['Parameters'])

## Score del mejor modelo

In [39]:
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')

MAE: 0.01113527118700206
MSE: 0.0001640036989003023
RMSE: 0.012806392891845162
R2: 0.996217476285399


## Guardar el mejor modelo

In [40]:
pickle.dump(best_model, open('model.pkl', 'wb'))